# Graph Logical

This file goes through all schools and adds logical knowledge to the graph by counting the number of public transportation and rating them on a scale from 1 to 6.

- 1 (best rating) if 5 or more public transportation methods nearby 300m, with at least 1 being a metro
- 2 if 5 public transporations menthods nearby 300m, but no metro included
- 3 if at least 5 public transportations are in the range of 600m
- 4 if at least 3 public transportations are in the range of 600m
- 5 if less than 3 public transporations are in the range of 600m
- 6 if no public transporation at all available


Running this notebook takes around 45min.

## Imports

In [1]:
from neomodel import db, config
from credentials import Credentials
from models import School

config.DATABASE_URL = Credentials.getNeo4JDatabaseURI()

## Add rating

In [18]:
def assign_school_ratings():
    all_schools = School.nodes.all()

    for school in all_schools:
        print(f'Checking school {school.name}...')

        nearby_stops_300m = school.is_nearby.all()
        nearby_stops_600m = school.is_in_area.all()
        
        stops_within_300m = 0
        stops_within_600m = 0
        has_metro_within_300m = False
        has_metro_within_600m = False
        
        line_count = {}

        def add_valid_line(line_name):
            if line_name not in line_count:
                line_count[line_name] = 0
            if line_count[line_name] < 1:
                line_count[line_name] += 1
                return True
            return False

        # Check each stop within 300m and exclude night buses
        for stop in nearby_stops_300m:
            print(f"Stop {stop.name} is within 300m of {school.name} with the following valid lines:")
            
            for line in stop.has_line:
                valid_stop = False
                if not line.name.startswith("N"):  # Exclude night buses
                    if add_valid_line(line.name):  # Ensure line is counted at most 1 time
                        print(line.name)
                        valid_stop = True
                        if line.name.startswith("U"):  # Check for metro
                            has_metro_within_300m = True
                if valid_stop:
                    stops_within_300m += 1

        # Check each stop within 600m and exclude night buses
        for stop in nearby_stops_600m:
            print(f"Stop {stop.name} is within 600m of {school.name} with the following valid lines:")
            
            for line in stop.has_line:
                valid_stop = False
                if not line.name.startswith("N"):  # Exclude night buses
                    if add_valid_line(line.name):  # Ensure line is counted at most 1 times
                        print(line.name)
                        valid_stop = True
                        if line.name.startswith("U"):  # Check for metro
                            has_metro_within_600m = True
                if valid_stop:
                    stops_within_600m += 1
        
        print(f'Stops within 300m: {stops_within_300m}. Stops within 600m: {stops_within_600m}. Metro: {has_metro_within_300m}')

        if (stops_within_300m >= 4 and has_metro_within_300m) or (stops_within_300m + stops_within_600m >= 6 and has_metro_within_300m) or (stops_within_300m + stops_within_600m >= 8):
            school.rating = 1
        elif (stops_within_300m >= 4 and not has_metro_within_300m) or (stops_within_300m + stops_within_600m >= 6):
            school.rating = 2
        elif (stops_within_600m + stops_within_300m >= 4)  or has_metro_within_300m or has_metro_within_600m:
            school.rating = 3
        elif (stops_within_600m + stops_within_300m >= 3):
            school.rating = 4
        elif stops_within_600m + stops_within_300m < 3 and stops_within_600m + stops_within_300m > 0:
            school.rating = 5
        else:
            school.rating = 6
        
        print(f'Rating for {school.name} is {school.rating}.\n\n')

        school.save()

assign_school_ratings()

Checking school GRg 3 Kundmanngasse 20-22...
Stop Geusaugasse is within 300m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
4A
Stop Kundmanngasse is within 300m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
Stop Rasumofskygasse is within 300m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
Stop Löwengasse is within 300m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
1
Stop Rochusgasse is within 300m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
74A
U3
Stop Apostelgasse is within 600m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
77A
Stop Hetzgasse is within 600m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
Stop Sechskrügelgasse is within 600m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
O
Stop Hintzerstraße is within 600m of GRg 3 Kundmanngasse 20-22 with the following valid lines:
Stop Weyrgasse is within 600m of GRg 3 Kundmanngasse 20-22 with the following valid lines